In [54]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)

In [60]:
df_train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")
df_train.phone = df_train.phone.fillna(-1).astype(np.int64)

In [61]:
df_train.head()

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,imquires@zoho.vo,typo,10111147
1,12172071-T0,ANDRÉ ALMEIDA BLANCO,officer,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,-1,NaN,typo,12172071
2,10104491-T1,MOONTA HOLDINGS LTD.,entity,NaN,65013464145,NaN,typo,10104491
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,consulting@zoho.de,original,10068808
4,13011715,GRIGORY UVAROV,officer,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,-1,NaN,original,13011715


In [66]:
df_train[df_train.linked_id == 10111147]

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,imquires@zoho.vo,typo,10111147
81089,10111147-M2,ALDIA ENTERPRISES LTD.,entity,NaN,-1,NaN,missing,10111147
115734,10111147,ALDIA ENTERPRISES LTD.,entity,NaN,4102971003602,inquires@zoho.it,original,10111147
169096,10111147-M0,ALDIA ENTERPRISES LTD.,entity,NaN,-1,NaN,missing,10111147
270623,10111147-T2,ALDIA ENTERPRISES LTD.,entity,NaN,4102971003602,inquires@zhho.it,typo,10111147
303571,10111147-M1,ALDIA ENTERPRISES LTD.,entity,NaN,4102971003602,inquires@zoho.it,missing,10111147


In [68]:
import editdistance
editdistance.eval(df_train.loc[0,'email'], df_train.loc[115734,'email'])

3

In [69]:
df_no_nans = df_train[~df_train.email.isna()]

In [70]:
df_no_nans

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,imquires@zoho.vo,typo,10111147
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,consulting@zoho.de,original,10068808
5,10105687,ODIN GROUP INVESTMENTS S.A.,entity,NaN,49668906658,info@outlook.cz,original,10105687
6,10016805-T2,CAPRI SOLUTIONS LIMITED,entity,NaN,-1,help@CAPRI.SOLUTIONS.LxMITED.de,typo,10016805
7,12133421,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",-1,CrossmatchSA@aol.gov,original,12133421
8,12126772,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,-1,support@LAURENCE.POUNTNEY.LTD.gov,original,12126772
11,10002433-NV0,COMMTECH FOUNDATION INC.,entity,1441 Brickell Avenue 1200 Four Seasons Tower M...,-1,consulting@yahoo.it,new_value,10002433
12,10164485,BROTHERHOOD FOUNDATION..,entity,NaN,14680064152,BROTHERHOODFOUNDATION@protonmail.co.uk,original,10164485
14,12188364,LI GUOBANG,officer,ROOM 102; BUILDING 37; NO.18; TAIPINGJIAO 4 RO...,-1,GUOBANG@icloud.com,original,12188364
16,10117892-T4,LONGXI INVESTMENT LTD.,entity,NaN,332588033405,info@outlook.com,typo,10117892


In [78]:
import string
import unidecode
def create_email_letters_matrix(df):
    df = df[['record_id','email']]
    df.email =  df.email.fillna('')
    df.email = df.email.astype(str) # convert to string
    df.email = df.email.str.lower() # lowercase
    no_accents = []
    for s in df.email:
        no_accents.append(unidecode.unidecode(s))
    df.email = no_accents
    # create return matrix
    columns = ['record_id','email','a','b','c','d','e','f','g','h','i','j','k','l',
               'm','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2',
              '3','4','5','6','7','8','9']
    email_letters_matrix = pd.DataFrame(columns=columns)
    email_letters_matrix.record_id = df.record_id.copy()
    email_letters_matrix.email = df.email.copy()
    # count occurence of each letter and add the columns to the return df
    for l in tqdm(['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                   'o','p','q','r','s','t','u','v','w','x','y','z','0','1','2',
                   '3','4','5','6','7','8','9']):
        new_col = []
        for (i,n) in zip(email_letters_matrix.index, email_letters_matrix.email):
            new_col.append(n.count(l))
        email_letters_matrix[l] = new_col
    return email_letters_matrix

In [79]:
create_email_letters_matrix(df_train)

,record_id,email,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,0,1,2,3,4,5,6,7,8,9
0,10111147-T1,imquires@zoho.vo,0,0,0,0,1,0,0,1,2,0,0,0,1,0,3,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,12172071-T0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10104491-T1,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10068808,consulting@zoho.de,0,0,1,1,1,0,1,1,1,0,0,1,0,2,3,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,13011715,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,10105687,info@outlook.cz,0,0,1,0,0,1,0,0,1,0,1,1,0,1,4,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,10016805-T2,help@capri.solutions.lxmited.de,1,0,1,2,3,0,0,1,3,0,0,3,1,1,2,2,0,1,2,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,12133421,crossmatchsa@aol.gov,3,0,2,0,0,0,1,1,0,0,0,1,1,0,3,0,0,1,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,12126772,support@laurence.pountney.ltd.gov,1,0,1,1,3,0,1,0,0,0,0,2,0,3,3,3,0,2,1,3,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,11002523,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
